In [1]:
# imports
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.cross_validation import cross_val_score

In [2]:
# set up folder link and files
folder_link = 'C:/Users/Michael/projects/kaggle/rossman/'

train_file = folder_link + 'train.csv'
test_file = folder_link + 'test.csv'
submission_file = folder_link + 'predictions_RandomForest.csv'
store_file = folder_link + 'store.csv'

In [3]:
# set up dataframes
train = pd.read_csv(train_file)
test = pd.read_csv(test_file)
store = pd.read_csv(store_file)

C:\Users\Michael\Anaconda\lib\site-packages\pandas\io\parsers.py:1170: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
# merge train and store data
train1 = train

# change state holiday to numeric
train1.StateHoliday = train1.StateHoliday.map({0:0,'0':1,'a':2,'b':3,'c':4})
train1.StateHoliday = train1.StateHoliday.astype(int)

In [5]:
# create features and response
feature_cols = ['DayOfWeek','Open','Promo','StateHoliday','SchoolHoliday']
train2 = train1.sample(n=100)
X = train2[feature_cols]
y = train2.Sales

In [6]:
# instantiate estimator, fit with all data
rfclf = RandomForestClassifier(n_estimators=100,max_features=3,oob_score=True,random_state=1)
rfclf.fit(X,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=3, max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=True, random_state=1, verbose=0, warm_start=False)

In [7]:
# compute the feature importances
pd.DataFrame({'feature':feature_cols,'importance':rfclf.feature_importances_})

,feature,importance
0,DayOfWeek,0.433157
1,Open,0.270074
2,Promo,0.100515
3,StateHoliday,0.103959
4,SchoolHoliday,0.092296


In [8]:
# remove NANs from test
test.loc[test.Open.isnull(),'Open'] = 1
test.loc[test.StateHoliday.isnull(),'StateHoliday'] = 0
test.StateHoliday = test.StateHoliday.map({0:0,'0':1,'a':2,'b':3,'c':4})
test.StateHoliday = test.StateHoliday.astype(int)

In [10]:
# predict values using random forests
y_pred = rfclf.predict(test[feature_cols])
y_pred

array([10498, 10498, 10498, ...,  2030,  2030,  8274], dtype=int64)

In [11]:
# create submission dataframe
submit = pd.DataFrame({'Id':test['Id'],'Sales':y_pred})

In [12]:
# create csv file
submit.to_csv(submission_file,index=False)